# Flatten Equivacard event data

In [ ]:
import plotnine
import pandas as pd
import numpy as np
import json
from datetime import datetime
import os

In [ ]:
import saga_py_aws
import saga_py_mongo

In [ ]:
secret = saga_py_aws.get_mongo_uri_secret('prod')
client = saga_py_mongo.get_mongo_client(secret)

In [ ]:
mapping_df = pd.read_csv('./data/cognition_mapping.csv', dtype= {'user_id': str})

In [ ]:
mapping_df

In [ ]:
query = [
    {"$match": {"activity":"EQUIVACARDS"}},
    {"$match": {"user_id": {"$in": list(mapping_df.user_id.values)}}}
     ]

In [ ]:
results = saga_py_mongo.aggregate_query_mongo(aggregate_query=query, collection_client=client['xlr8_beta']['analytics_simple_event_data'])

In [ ]:
event_json = results

In [ ]:
results[0:10]

In [ ]:
example_id='1339'
example= [result for result in results if result['user_id']==example_id]

In [ ]:
len(example)

In [ ]:
example_id='1339'
example2= [result for result in event_json if result['user_id']==example_id]

In [ ]:
len(example2)

## Preview event shape

In [ ]:
event_json[0:3]

In [ ]:
event_json[-1]

In [ ]:
def unnest_dict(event):
    event_copy = event.copy()
    if type(event) == dict:
        keys = event.keys()
        for key in keys:
            if type(event_copy[key]) == dict:
                event_copy.update(unnest_dict(event_copy[key]))
                del event_copy[key]
    return event_copy
    

In [ ]:
unnest_dict(event_json[4])

In [ ]:
event_json[4]

In [ ]:
flat_events = []
for event in event_json:
    flat_events.append(unnest_dict(event))

In [ ]:
json_df = pd.json_normalize(flat_events, meta=['_id.$oid'])
json_df.head()

In [ ]:
json_df.sample(1).to_dict('records')

## Align timestamps

In [ ]:
json_df.server_timestamp.values[0]

In [ ]:
json_df['server_timestamp'].describe()

In [ ]:
json_df['timestamp'].describe()

In [ ]:
print(datetime.utcfromtimestamp(1.662995e+15/1000000).strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
print(datetime.utcfromtimestamp(1.662995e+09).strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
print(datetime.utcfromtimestamp(json_df.server_timestamp.min()/1000).strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
print(datetime.utcfromtimestamp(json_df.server_timestamp.max()/1000).strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
def align_unix_convention(x):
    if not np.isnan(x.server_timestamp):
        return x.server_timestamp/1000 
    else:
        ts = x.timestamp
        above_12 = np.floor(np.log10(ts)-9)
        return ts/(10**(above_12))
    

In [ ]:
print(json_df.head(1).apply(lambda x: align_unix_convention(x), axis=1))

In [ ]:
json_df['unix_timestamp_combined'] = json_df.apply(lambda x: align_unix_convention(x), axis=1)

In [ ]:
json_df['timestamp_combined'] = json_df.apply(lambda x: datetime.utcfromtimestamp(x.unix_timestamp_combined), axis=1)

In [ ]:
json_df['unix_timestamp_combined'].describe()

In [ ]:
print(json_df.timestamp_combined.min())

In [ ]:
print(json_df.timestamp_combined.max())

In [ ]:
json_df.sample(5)

## Review event distribution across field categories

In [ ]:
json_df.groupby('activity').count()

In [ ]:
json_df.groupby('event_name').count()

In [ ]:
json_df.groupby('bucket').count()

In [ ]:
json_df.groupby('activity')['user_id'].nunique()

In [ ]:
equiv_events_df = json_df[json_df.activity.isin(['EQUIVACARDS'])]

In [ ]:
equiv_events_df.shape

In [ ]:
equiv_events_df.columns

## Flatten object columns

In [ ]:
equiv_events_df.sample(5)

In [ ]:
equiv_events_df[equiv_events_df.best_play.notna()].sample(5).best_play.values

In [ ]:
equiv_events_df.best_play

`best_play` is a complex object and would require targeted processing to pull out value

Length of the best play and first move of the best play seem likely useful 

In [ ]:
best_play_df = equiv_events_df.best_play.apply(pd.Series) 

In [ ]:
best_play_df.columns = [f"best_play_turn_{item}" for item in best_play_df.columns]

In [ ]:
best_play_df

In [ ]:
best_play_df[best_play_df.best_play_turn_0.notna()].head()

In [ ]:
best_play_df[best_play_df.best_play_turn_0.notna()].head()

In [ ]:
best_play_0_df = pd.json_normalize(best_play_df.best_play_turn_0,errors='ignore')

In [ ]:
best_play_0_df

In [ ]:
best_play_0_df.columns = [f"best_play_turn_0_{item}" for item in best_play_0_df.columns]

In [ ]:
equiv_events_df[equiv_events_df.board.notna()].sample(5).board.values

In [ ]:
board_df = equiv_events_df.board.apply(pd.Series) 

In [ ]:
board_df.columns = ["board_left_card", "board_right_card"]

In [ ]:
board_df[board_df.board_right_card.notna()].sample(5)

In [ ]:
equiv_events_df[equiv_events_df.p1_hand.notna()].sample(5).p1_hand.values

In [ ]:
p1_hand_df = equiv_events_df.p1_hand.apply(pd.Series) 

In [ ]:
p1_hand_df.columns = [f"p1_hand_card_{item}" for item in p1_hand_df.columns]

In [ ]:
p1_hand_df['p1_hand_size'] = p1_hand_df.count(axis=1)

In [ ]:
p1_hand_df[p1_hand_df.p1_hand_card_0.notna()].sample(5)

In [ ]:
equiv_flat_df = pd.concat([
    equiv_events_df.reset_index(drop=True), 
    best_play_df.reset_index(drop=True),
    best_play_0_df.reset_index(drop=True),
    board_df.reset_index(drop=True),
    p1_hand_df.reset_index(drop=True)
], axis=1)

In [ ]:
equiv_flat_df.sample(5)

## Correct Connect the Drops labels

In [ ]:
game_temp_df = equiv_flat_df.sort_values(by='timestamp_combined')
# [equiv_flat_df.event_name.isin(["launched_connect_the_drops", "launched_equivacards", "user_won", "user_lost"])]
# .sort_values(by='timestamp_combined')

In [ ]:
equiv_flat_df

In [ ]:
game_temp_df

In [ ]:
game_temp_df = equiv_flat_df.sort_values(by='timestamp_combined')

In [ ]:
game_temp_df['game_launch'] = game_temp_df.event_name.apply(lambda x: x in ['launched_connect_the_drops', 'launched_equivacards'])

In [ ]:
game_temp_df['game_start'] = game_temp_df.event_name.apply(lambda x: x in ['launched_equivacards', 'initial_game_state', 'play_again_yes'])

In [ ]:
game_temp_df['game_start_time'] = game_temp_df.apply(lambda x: x.timestamp_combined if x.game_start else None , axis=1 )

In [ ]:
def correct_launch_activity(x):
    if x == 'launched_connect_the_drops':
        return "CONNECT_THE_DROPS"
    elif x ==  'launched_equivacards':
        return "EQUIVACARDS"

In [ ]:
game_temp_df['corrected_activity'] = game_temp_df.event_name.apply(correct_launch_activity)

In [ ]:
game_temp_df['game_end'] = game_temp_df.event_name.apply(lambda x: x in ['user_won', 'user_lost'])

In [ ]:
game_temp_df['game_end_time'] = game_temp_df.apply(lambda x: x.timestamp_combined if x.game_end else None , axis=1 )

In [ ]:
game_temp_df['user_turn_start'] = game_temp_df.event_name.apply(lambda x: x in ['user_turn'])

In [ ]:
game_temp_df['user_turn_start_time'] = game_temp_df.apply(lambda x: x.timestamp_combined if x.user_turn_start else None , axis=1 )

In [ ]:
game_temp_df['user_turn_end'] = game_temp_df.event_name.apply(lambda x: x in ['user_drew_card', 'user_won', 'user_lost'])

In [ ]:
game_temp_df['user_turn_end_time'] = game_temp_df.apply(lambda x: x.timestamp_combined if x.user_turn_end else None , axis=1 )

In [ ]:
game_temp_df['user_took_action'] = game_temp_df.event_name.apply(lambda x: x in ['user_played_card', 'user_drew_card', 'play_not_allowed'])

In [ ]:
game_temp_df['user_action_time'] = game_temp_df.apply(lambda x: x.timestamp_combined if x.user_took_action else None , axis=1 )

In [ ]:
game_temp_df['user_launch_index'] = game_temp_df.groupby('user_id').game_launch.cumsum()
game_temp_df['user_game_index'] = game_temp_df.groupby('user_id').game_start.cumsum()

In [ ]:
game_temp_df['user_turn_start_index'] = game_temp_df.groupby(['user_id','user_game_index']).user_turn_start.cumsum()
game_temp_df['user_turn_end_index'] = game_temp_df.groupby(['user_id','user_game_index']).user_turn_end.cumsum()

In [ ]:
game_temp_df['turn_id'] = game_temp_df.apply(lambda x: '-'.join([str(x.user_turn_start_index), str(x.user_turn_end_index)]), axis=1)

In [ ]:
game_temp_df['match_type'] = game_temp_df.apply(lambda x: 'color' if x.match_color is True else 'value' if x.match_value is True else 'algebraic' if x.match_algebraic is True else None, axis=1)

In [ ]:
game_temp_df.groupby(['user_id','user_game_index'])[['match_type']]

In [ ]:
game_temp_df['last_match_type'] = game_temp_df.groupby(['user_id','user_game_index', 'turn_id', 'event_name']).match_type.shift(1)

In [ ]:
game_temp_df['match_type_change'] = game_temp_df.apply(lambda x: x.match_type != x.last_match_type if type(x.last_match_type) == str else None, axis=1)

In [ ]:
game_temp_df['comp_turn'] = game_temp_df.apply(lambda x: x.user_turn_start_index == x.user_turn_end_index, axis=1)
game_temp_df['user_turn'] = ~game_temp_df['comp_turn']

In [ ]:
game_temp_df[game_temp_df.user_id == '4492'][['timestamp_combined','event_name', 'corrected_activity', 'game_end', 'game_start', 'user_launch_index', 'user_game_index', 'user_turn_start_time', 'user_turn_end_time', 'turn_id', 'user_turn_start_index', 'match_type', 'last_match_type', 'match_type_change', 'match_color', 'match_value', 'match_algebraic']].to_csv('./4492.csv')

In [ ]:
game_temp_df[game_temp_df.user_id == '1336'][['timestamp_combined','event_name', 'user_game_index', 'user_turn_start', 'user_turn_end', 'user_turn_start_index', 'user_turn_end_index', 'turn_id', 'comp_turn', 'user_turn', 'game_end_time', 'game_start_time', 'user_turn_start_index']].to_csv('./1336.csv')

In [ ]:
def correct_activity(x):
    if x == 'launched_connect_the_drops':
        return "CONNECT_THE_DROPS"
    elif x ==  'launched_equivacards':
        return "EQUIVACARDS"

In [ ]:
game_temp_df['corrected_activity'] = game_temp_df.event_name.apply(lambda x: correct_activity(x))

In [ ]:
game_by_launch_df = game_temp_df[game_temp_df['corrected_activity'].notna()][['user_id', "user_launch_index","corrected_activity"]]

In [ ]:
game_by_launch_df.sample(5)

In [ ]:
corrected_activity_events_df = pd.merge(game_temp_df.drop('corrected_activity', axis=1), game_by_launch_df, on=["user_id", "user_launch_index"])

In [ ]:
corrected_activity_events_df.groupby('corrected_activity')['user_id'].nunique()

In [ ]:
corrected_equivacards_events = corrected_activity_events_df[corrected_activity_events_df.corrected_activity=="EQUIVACARDS"]

In [ ]:
assert corrected_equivacards_events[corrected_equivacards_events.user_id=='4492'].user_game_index.max()==52

In [ ]:
assert corrected_equivacards_events[corrected_equivacards_events.user_id=='1336'].user_game_index.max()==9

In [ ]:
corrected_equivacards_events.describe()

In [ ]:
corrected_equivacards_events.groupby('event_name').count()

In [ ]:
corrected_equivacards_events.user_game_index.max()

In [ ]:
corrected_equivacards_events.groupby('event_name')['user_id'].nunique()

In [ ]:
corrected_equivacards_events.dtypes

In [ ]:
corrected_equivacards_events.columns.sort_values()

In [ ]:
corrected_equivacards_events.shape

In [ ]:
corrected_equivacards_events.shape

In [ ]:
corrected_equivacards_events.columns

In [ ]:
corrected_equivacards_events[corrected_activity_events_df.user_id == '4492']

In [ ]:
corrected_equivacards_events[corrected_activity_events_df.user_id == '4492'][['event_name', 'corrected_activity', 'game_end', 'game_start', 'user_launch_index', 'user_game_index', 'match_type']].to_csv('4492.csv')

# Calc number cards

In [ ]:
example_df = corrected_equivacards_events[corrected_activity_events_df.user_id.isin(
    [
        '1336'
#         , '4492'
    ])].copy()

In [ ]:
corrected_equivacards_events.columns

In [ ]:
example_df[(example_df.user_id == '1336') & (example_df.user_game_index == 4)].to_csv('example_events.csv')

In [ ]:
example_df[['user_launch_index', 'user_game_index', 'turn_id', 'event_name','user_id', 'board_right_card', 'board_left_card','p1_hand', 'card', 'value', 'match_color','match_value', 'match_algebraic']].to_csv('example_events.csv')

In [ ]:
example_df[(example_df.user_id == '1336') & (example_df.user_game_index == 5)][['user_launch_index', 'user_game_index', 'user_turn_start_index', 'turn_id', 'event_name','user_id', 'board_right_card', 'board_left_card','p1_hand', 'card', 'value', 'match_color','match_value', 'match_algebraic']].to_csv('example_events.csv')

In [ ]:
example_df.groupby(['user_id', 'user_game_index', 'turn_id', 'user_turn', 'event_name']).event_name.count().to_csv('example_by_turn.csv')

In [ ]:
example_df.groupby(['user_id', 'user_game_index', 'event_name']).event_name.count().to_csv('example_by_game.csv')

## Turn Metrics

In [ ]:
example_df[(example_df.user_id == '1336') & (example_df.user_game_index == 5) & (example_df.user_turn_start_index == 1)][['user_turn_start_time', 'user_action_time']]

In [ ]:
# example_df['num_of_cards_played_in_turn'] = 
example_df[example_df.event_name=='user_played_card'].groupby(['user_id', 'user_game_index', 'user_turn_start_index']).card.count().reset_index()

In [ ]:
example_df.match_type_change.sum?

In [ ]:
# example_df['category_match_switches'] = 
example_df.groupby(['user_id', 'user_game_index', 'user_turn_start_index']).match_type_change.sum().reset_index().astype(int)

In [ ]:
example_df

In [ ]:
example_df.groupby(['user_id', 'user_game_index', 'user_turn_start_index']).match_type.first()

In [ ]:
example_df[(example_df.user_id == '1336') & (example_df.user_game_index == 2) & (example_df.user_turn_start_index == 1)][['user_turn_start_time', 'user_action_time']]

In [ ]:
example_df[(example_df.user_id == '1336') & (example_df.user_game_index == 2) & (example_df.user_turn_start_index == 1)][['user_id', 'user_turn_start_time', 'user_action_time']].groupby(['user_id']).apply(lambda x: (x.user_action_time.min() - x.user_turn_start_time.min()).total_seconds())

In [ ]:
(pd.Timestamp('2022-08-16 19:39:01.933') - pd.Timestamp('2022-08-16 19:38:52.663')).total_seconds()

In [ ]:
# example_df['time_to_first_move_seconds'] = 
example_df.groupby(['user_id', 'user_game_index', 'user_turn_start_index']).apply(lambda x: (x.user_action_time.min() - x.user_turn_start_time.min()).total_seconds())

In [ ]:
# example_df['time_to_first_move_seconds'] = 
example_df.groupby(['user_id', 'user_game_index', 'user_turn_start_index']).apply(lambda x: (x.user_action_time.min() - x.user_turn_start_time.min()).microseconds)

In [ ]:
# example_df['turn_time_seconds'] = 
example_df.groupby(['user_id', 'user_game_index', 'user_turn_start_index']).apply(lambda x: (x.user_turn_end_time.max() - x.user_turn_start_time.min()).microseconds/100000)

In [ ]:
# example_df['turn_time_seconds'] = 


In [ ]:
# example_df['num_of_cards_played'] = 
example_df.groupby(['user_id', 'user_game_index', 'user_turn_start_index']).apply(lambda x: x.user_turn_start_time.min() )

In [ ]:
example_df.user_turn_end_time.max() 
# - example_df.user_turn_end_time.min()

In [ ]:
example_df.user_turn_end_time.min()

In [ ]:
example_df.user_turn_end_time.max()

In [ ]:
# example_df['num_of_cards_played'] = 
example_df.groupby(['user_id', 'user_game_index', 'turn_id', 'user_turn', 'event_name']).card.count()

# Calc options

In [ ]:
calc_df = example_df[
    example_df.event_name=='game_state_changed'
][[
    'user_launch_index', 'user_game_index','turn_id', 
    'user_turn', 'event_name','user_id', 'board_right_card', 'board_left_card', 'p1_hand_card_0',
    'p1_hand_card_1', 'p1_hand_card_2', 'p1_hand_card_3', 'p1_hand_card_4', 
    'p1_hand_card_5','p1_hand_card_6', 'p1_hand_card_7', 'p1_hand_card_8', 'p1_hand_card_9',
       'p1_hand_card_10', 'card', 'value', 'match_color','match_value', 'match_algebraic']].copy()

In [ ]:
p1_card_columns = [column for column in calc_df.columns if 'p1_hand_card' in column]
p1_card_columns

In [ ]:
for p1_column in p1_card_columns:
    

In [ ]:
calc_df.to_csv('example_events.csv')

In [ ]:
def eval_card(x,expr):
    if expr=='1':
        return(1)
    elif expr=='2':
        return(2)
    elif expr=='3':
        return(3)
    elif expr=='4':
        return(4)
    elif expr=='5':
        return(5)
    elif expr=='6':
        return(6)
    elif expr=='7':
        return(7)
    elif expr=='8':
        return(8)
    elif expr=='x':
        return(x)
    elif expr=='x+x':
        return(x+x)
    elif expr=='x+1':
        return(x+1)
    elif expr=='x+2':
        return(x+2)
    elif expr=='9-x':
        return(9-x)
    elif expr=='2x':
        return(2*x)
    elif '=' in expr:
        return False
    print(f'unexpected card expression {expr}')

In [ ]:
calc_df['board.x_val'] = calc_df.apply(lambda x: int(x.board_left_card[-1:]), axis=1)

In [ ]:
calc_df['board.color'] = calc_df.apply(lambda x: (x.board_right_card.split(".")[0]), axis=1)

In [ ]:
calc_df['board.right_val'] = calc_df.apply(lambda x: (x.board_right_card.split(".")[1]), axis=1)

In [ ]:
calc_df['board.right_val'] = calc_df.apply(lambda x: (x.board_right_card.split(".")[1]), axis=1)

In [ ]:
calc_df['aval_value'] = calc_df.apply(lambda x: eval_card(int(x['board.x_val']), x['board.right_val']), axis =1)

In [ ]:
for column in p1_card_columns:
    calc_df[f'{column}_can_match_color'] = calc_df.apply(lambda x: x['board.color'] in str(x[column]), axis =1)
    calc_df[f'{column}_can_match_value'] = calc_df.apply(lambda x: x['board.right_val'] in str(x[column]), axis =1)
    

In [ ]:
for column in p1_card_columns:
    calc_df[f'{column}_eval_value'] = calc_df.apply(lambda x: eval_card(int(x['board.x_val']), x[column].split(".")[1]) if type(x[column])==str else None, axis =1)
    

In [ ]:
for column in p1_card_columns:
    calc_df[f'{column}_can_match_algebraic'] = calc_df.apply(
        lambda x: str(x[f'{column}_eval_value']) == str(x['aval_value'])  if type(x[column])==str else None, axis =1)
    

In [ ]:
can_match_columns={}
for match_type in ['algebraic', 'color', 'value']:
    can_match_columns[match_type] = [ column+ f'_can_match_{match_type}' for column in p1_card_columns]
    calc_df[f'num_can_match_{match_type}'] = calc_df[can_match_columns[match_type]].sum(axis=1)

In [ ]:
pd.merge(example_df, calc_df, how='outer').to_csv('./example_events_with_options.csv')

In [ ]:
####

## Review of game time and distribution of events per game 

In [ ]:
from plotnine import ggplot, geom_point, aes, stat_smooth, facet_wrap, scale_x_date, geom_line,facet_grid, theme, element_text, labs, element_blank, ggtitle, geom_bar

In [ ]:
(ggplot(
   corrected_equivacards_events, aes('timestamp_combined', 'factor(user_id)', color= 'factor(user_id)'))
+ geom_point(show_legend=False)
 + geom_line(show_legend=False)
  + labs(x= "date", y="User Id")
 + ggtitle("Game by user_id vs date")
 + theme(figure_size=(6, 4), axis_text_x=element_text(rotation=90, hjust=1)) 
)

In [ ]:
(ggplot(
   corrected_equivacards_events, aes('user_game_index', fill= 'factor(event_name)'))
+ geom_bar(stat='count', position= 'stack')
 + facet_grid('user_id ~', scales ='free', )
 + theme(figure_size=(4, 10), axis_text_x=element_text(rotation=90, hjust=1), strip_text_y = element_text(angle = 0)) 
 + labs(x= "User Game Index", y="Event count")
 + ggtitle("Game events by user_id")
)

## Cursory comparison of best move 0 to taken move 0

In [ ]:
corrected_equivacards_events['previous_best_move_0'] = corrected_equivacards_events.groupby('user_id').best_play_turn_0_label.shift(2).apply(lambda x: x.replace(',', '.') if type(x)==str else None ) 
corrected_equivacards_events['made_best_move_0'] = corrected_equivacards_events.apply(lambda x: x.previous_best_move_0==x.card, axis =1) 

In [ ]:
corrected_equivacards_events[['event_name','card', 'best_play_turn_0_label', 'previous_best_move_0', 'made_best_move_0']].head(15)

In [ ]:
corrected_equivacards_events[corrected_equivacards_events.event_name=='user_turn'].groupby(['user_id', 'made_best_move_0'])['$oid'].count()

In [ ]:
corrected_equivacards_events[corrected_equivacards_events.event_name.isin(['user_won', 'user_lost'])].groupby(['user_id', 'event_name'])['$oid'].count()

In [ ]:
corrected_equivacards_events[corrected_equivacards_events.event_name.isin(['user_won', 'user_lost'])].groupby(['event_name'])['$oid'].count()

In [ ]:
corrected_equivacards_events[corrected_equivacards_events.event_name.isin(['user_turn'])].groupby(['user_game_index'])['$oid'].count()

In [ ]:
corrected_equivacards_events.groupby('deck_version').count()